This notebook explores dependency parsing by identifying the actions and objects that are characteristically associated with male and female characters.

In [1]:
import spacy, math
from collections import Counter
import operator

In [2]:
nlp = spacy.load('en_core_web_sm')

We'll run seven novels by Jane Austen through spacy (this will take a few minutes).

In [3]:
filenames=["../data/fiction/emma.txt", "../data/fiction/lady_susan.txt", "../data/fiction/mansfield_park.txt", "../data/fiction/northanger_abbey.txt", "../data/fiction/persuasion.txt", "../data/fiction/pride.txt", "../data/fiction/sense_and_sensibility.txt"]
all_tokens=[]
for filename in filenames:
    print(filename)
    data=open(filename, encoding="utf-8").read()
    tokens=nlp(data)
    all_tokens.extend(tokens)

../data/fiction/emma.txt
../data/fiction/lady_susan.txt
../data/fiction/mansfield_park.txt
../data/fiction/northanger_abbey.txt
../data/fiction/persuasion.txt
../data/fiction/pride.txt
../data/fiction/sense_and_sensibility.txt


In [4]:
print (len(all_tokens))

972808


In [5]:
def test(maleCounter, femaleCounter, display=25):
    
    """ Function that takes two Counter objects as inputs and prints out a ranked list of terms
    more characteristic of the first counter than the second.  Here we'll use log-odds
    with an uninformative prior (from Monroe et al 2008, "Fightin Words", eqn. 22) as our metric.
    
    """
    
    vocab=dict(maleCounter) 
    vocab.update(dict(femaleCounter))
    maleSum=sum(maleCounter.values())
    femaleSum=sum(femaleCounter.values())

    ranks={}
    alpha=0.01
    alphaV=len(vocab)*alpha
        
    for word in vocab:
        
        log_odds_ratio=math.log( (maleCounter[word] + alpha) / (maleSum+alphaV-maleCounter[word]-alpha) ) - math.log( (femaleCounter[word] + alpha) / (femaleSum+alphaV-femaleCounter[word]-alpha) )
        variance=1./(maleCounter[word] + alpha) + 1./(femaleCounter[word] + alpha)
        
        ranks[word]=log_odds_ratio/math.sqrt(variance)

    sorted_x = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)
    
    print("Most male:")
    for k,v in sorted_x[:display]:
        print("%.3f\t%s" % (v,k))
    
    print("\nMost female:")
    for k,v in reversed(sorted_x[-display:]):
        print("%.3f\t%s" % (v,k))

Spacy uses the [ClearNLP dependency labels](https://github.com/clir/clearnlp-guidelines/blob/master/md/specifications/dependency_labels.md), which are very close to the Stanford typed dependencies.  See the [Stanford dependencies manual](https://nlp.stanford.edu/software/dependencies_manual.pdf) for more information about each tag.  Parse information is contained in the spacy token object; see the following for which attributes encode the token text, idx (position in sentence), part of speech, and dependency relation.  The syntactic head for a token is another token given in `token.head` (where all of those same token attributes are accessible).

In [6]:
testDoc=nlp("He started his car.")
for token in testDoc:
    print("%s\t%s\t%s\t%s\t%s\t%s\t%s" % (token.text, token.idx, token.tag_, token.dep_, token.head.text, token.head.idx, token.head.tag_))


He	0	PRP	nsubj	started	3	VBD
started	3	VBD	ROOT	started	3	VBD
his	11	PRP$	poss	car	15	NN
car	15	NN	dobj	started	3	VBD
.	18	.	punct	started	3	VBD


Q1: Find the verbs that men are more characteristically the *subject* of than women.  Feel free to only consider subjects that are "he" and "she" pronouns.  This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" (`maleCounter`) and "she" (`femaleCounter`) as its syntactic subject.

In [7]:
def count_subjects():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "nsubj":
            if token.text.lower() == "he":
                maleCounter[token.head.lemma_]+=1
            elif token.text.lower() == "she":
                femaleCounter[token.head.lemma_]+=1
    
    return maleCounter, femaleCounter

In [8]:
male, female=count_subjects()
test(male, female, display=10)

Most male:
8.209	come
4.961	reply
4.701	say
4.212	seem
3.852	tell
3.768	leave
3.601	talk
3.148	marry
3.087	take
2.955	ask

Most female:
-7.886	feel
-5.497	see
-4.575	hear
-3.791	cry
-3.014	help
-2.849	expect
-2.796	resolve
-2.784	find
-2.744	remember
-2.697	believe


Q2: Find the verbs that men are more characteristically the *object* of than women.  Feel free to only consider objects that are "him" and "here" pronouns.  This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" (`maleCounter`) and "she" (`femaleCounter`) as its syntactic direct object. 

In [9]:
def count_objects():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "dobj":
            if token.text.lower() == "him":
                maleCounter[token.head.lemma_]+=1
            elif token.text.lower() == "her":
                femaleCounter[token.head.lemma_]+=1
    
    return maleCounter, femaleCounter

In [10]:
male, female=count_objects()
test(male, female, display=10)

Most male:
5.664	see
4.630	like
3.157	know
2.662	send
2.585	introduce
2.585	wish
2.357	recommend
2.329	suspect
2.124	believe
1.823	dislike

Most female:
-3.729	leave
-2.780	attend
-2.392	prevent
-2.378	please
-2.299	strike
-2.058	oblige
-2.022	treat
-1.960	support
-1.960	amuse
-1.895	escape


Q3: Find the objects that are *possessed* more frequently by men than women.  Feel free to only consider possessors that are "his" and "her" pronouns.   This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given term is possessed by "he" (`maleCounter`) and "she" (`femaleCounter`).

In [11]:
def count_possessions():
    maleCounter=Counter()
    femaleCounter=Counter()

    for token in all_tokens:
        if token.dep_ == "poss":
            if token.text.lower() == "his":
                maleCounter[token.head.lemma_]+=1
            elif token.text.lower() == "her":
                femaleCounter[token.head.lemma_]+=1
    
    return maleCounter, femaleCounter

In [12]:
male, female=count_possessions()
test(male, female, display=10)

Most male:
4.615	return
4.608	manner
4.178	house
3.922	name
3.807	horse
3.709	address
3.367	son
3.348	character
3.344	behaviour
3.333	attachment

Most female:
-7.128	mother
-5.353	aunt
-4.129	uncle
-3.903	spirit
-3.796	sister
-3.758	eye
-3.699	heart
-3.564	room
-3.043	thought
-3.041	feeling


Q4: Find the actions that are men do *to women* more frequently than women do *to men*.  Feel free to only consider subjects and objects that are "she"/"he"/"her"/"him" pronouns.   This function should return two Counter objects (`maleCounter` and `femaleCounter`) which counts the number of times a given verb has "he" as the subject and "her" as the object (`maleCounter`) and "she" as the subject and "him" as the object (`femaleCounter`).

In [13]:
def count_SVO_tuples():
    maleCounter=Counter()
    femaleCounter=Counter()

    dobj_verbs={}
    nsubj_verbs={}

    for token in all_tokens:
        if token.dep_ == "dobj":
            if token.text.lower() == "him" or token.text.lower() == "her":
                head=token.head
                if head not in dobj_verbs:
                    dobj_verbs[head]=[]
                dobj_verbs[head].append(token)
                
                
        if token.dep_ == "nsubj":
            if token.text.lower() == "he" or token.text.lower() == "she":
                head=token.head
                if head not in nsubj_verbs:
                    nsubj_verbs[head]=[]
                nsubj_verbs[head].append(token)

    for head_verb in dobj_verbs:
        if head_verb in nsubj_verbs:
            
            for subjectToken in nsubj_verbs[head_verb]:
                for objectToken in dobj_verbs[head_verb]:
                    if subjectToken.text.lower() == "he" and objectToken.text.lower() == "her":
                        maleCounter[head_verb.lemma_]+=1
                    elif subjectToken.text.lower() == "she" and objectToken.text.lower() == "him":
                        femaleCounter[head_verb.lemma_]+=1

                
    return maleCounter, femaleCounter

In [14]:
male, female=count_SVO_tuples()
test(male, female, display=10)

Most male:
3.174	tell
1.946	leave
1.566	assure
1.496	give
1.344	join
1.235	ask
1.108	love
1.083	admire
1.047	forget
0.652	call

Most female:
-3.182	see
-1.753	have
-1.161	entreat
-0.870	find
-0.870	understand
-0.855	thank
-0.713	like
-0.693	follow
-0.674	accept
-0.632	refuse
